In [37]:
# ******************************************************************************************
# courseProject.ipynb
# Justin Trubela, Austin Wolcott
#
 
import sqlite3
from sqlite3 import Error
 
 
def Create_student(conn, sID, sName):
    sql2 = "SELECT MAX(sID) FROM Students"
    cursor = conn.cursor()
    cursor.execute(sql2)
    sID = cursor.fetchall()[0]
    sID = int(sID[0])+1
    
    sql = "INSERT INTO Students (sID,sName) VALUES (?,?);"
    values = (sID,sName)
    cursor = conn.cursor()
    cursor.execute(sql, values)
    conn.commit()
    #userChoice = input("\nEnter a command:")
    student_menu(sID)
 
def Get_studentsName(conn, sID):
    sql = "SELECT sName FROM Students WHERE sID = ?"
    cursor = conn.cursor()
    cursor.execute(sql, (sID,))
    sName = cursor.fetchall()[0]
    return str(sName).replace("('","").replace("'.)","")
 
#----------------------------------------------------------------------------------------------------
 
def Create_course(conn, cID, cName, numCredits):
    sql = "INSERT INTO Courses (cID, cName, numCredits) VALUES (?,?,?);"
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
 
def Check_if_cID_exists(conn, cID):
    sql = "SELECT count(*) FROM Courses WHERE cID = ?"
    cursor = conn.cursor()
    cursor.execute(sql, (cID,))
    if(cursor.fetchall()[0] > (0,)):
        return True
    else:
        return False
 
def Check_if_Course_table_is_empty(conn):
    sql = "SELECT count(*) FROM Courses"
    cursor = conn.cursor()
    cursor.execute(sql)
    if(cursor.fetchall() == (0,)):
        return True
    else:
        return False
 
#----------------------------------------------------------------------------------------------------
 
def Show_all_Courses(conn, sID):
    sql = "SELECT * FROM Courses c"
    cursor = conn.cursor()
    cursor.execute(sql)
    numCourses = cursor.fetchall()
 
    if len(numCourses) < 1: 
        print("no courses available")
    else:
        print("Courses available: \n")
        print("  course ID ----- Course Name ----- numCredits  \n")
        for course in numCourses:
            print(course)
            print("\n")
 
def Show_all_My_Courses(conn, sID):
    sql = "SELECT c.cID, c.cName, c.numCredits FROM Courses c, Enrollment e WHERE c.cID = e.cID AND e.sID = ?; "
    cursor = conn.cursor()
    cursor.execute(sql, (sID,))
    numCourses = cursor.fetchall()
 
    if len(numCourses) == 0: 
        print("Student is not enrolled in any classes\n")
    else:
        print("You are Enrolled in the following classes: \n")
        print("  course ID ----- Course Name ----- numCredits  \n")
        for course in numCourses:
            print(course)
            print("\n")
 
def Show_similar_courses(conn, string):
    #take substring entered from the user and check
    sql = "SELECT * FROM Courses WHERE cName LIKE \"%?%\""
    cursor = conn.cursor()
    cursor.execute(sql.replace("?", str(string)))
    numCourses = cursor.fetchall()
 
    if len(numCourses) == 0: 
        print("no courses with those letters seem to be present")
    else:
        print("Courses found with those letters: \n")
        print("  course ID ----- Course Name ----- numCredits  \n")
        for course in numCourses:
            print(course)
            print("\n")
 
#----------------------------------------------------------------------------------------------------
 
#Student Menu: 
#Application starts by requesting student’s ID. If (-1) is introduced, a new student is created, and 
#the user is prompted for all necessary information. The main menu is the following: 
 
#L – List: lists all records in the course table 
#E – Enroll: enrolls the active student in a course; user is prompted for course ID; check for 
#conflicts, i.e., student cannot enroll twice in same course 
 
#W – Withdraw: deletes an entry in the Enrolled table corresponding to active student; student 
#is prompted for course ID to be withdrawn from 
#S – Search: search course based on substring of course name which is given by user; list all 
#matching courses 
#M – My Classes: lists all classes enrolled in by the active student. 
#X – Exit: exit application     
 
#----------------------------------------------------------------------------------------------------
 
def create_enrollment(conn, sID, cID):
    sql = "INSERT INTO Enrollment (sID,cID) VALUES (?,?);"
    values = (sID,cID)
    cursor = conn.cursor()
    cursor.execute(sql, values)
    conn.commit()
 
def remove_enrollment(conn, sID, cID):
    sql = "DELETE FROM Enrollment WHERE sID = ? AND cID = ?"
    values = (sID,cID)
    cursor = conn.cursor()
    try: 
        cursor.execute(sql, values)
        conn.commit()
    except Error as e:
        print(e)
    return conn
 
    cursor.execute(sql, values)
    conn.commit()
 
def check_if_sID_exists(conn, sID):
    sql = "SELECT count(*) FROM Students WHERE sID = ?"
    cursor = conn.cursor()
    cursor.execute(sql, (sID,))
    if(cursor.fetchall()[0] > (0,)):
        return True
    else:
        return False
 
def check_if_sID_is_enrolled_in_cID(conn, sID, cID):
    sql = "SELECT count(*) FROM Enrollment WHERE sID = ? AND cID = ?"
    values = (sID, cID)
    cursor = conn.cursor()
    cursor.execute(sql,values)
    if(cursor.fetchall()[0] > (0,)):
        return True
    else:
        return False
 
def student_menu(sID):
    while True:
        choice = input("\nEnter a command:")
        if choice == 'L' or choice == 'l':
            if (Check_if_Course_table_is_empty(conn) == False):
                Show_all_Courses(conn, sID)
                continue
        elif choice == 'E' or choice == 'e':
            while True: 
                cID = input(("Enter the course ID you wish to register in: "))
                #check if cID exists before registering
                #if not available
                if Check_if_cID_exists(conn, cID) == False:
                    print("invalid course ID entered. \n")
                    continue
                #if course ID is available
                else:
                    #Check if sID already registerd in chosen cID
                    if (check_if_sID_is_enrolled_in_cID(conn, sID, cID) == False):
                        create_enrollment(conn, sID, cID)
                        break
                    else:
                        print("This studentID is already registered for this course. Try another one")
                        continue
            continue
        elif choice == 'W' or choice == 'w':
            print("Which course would you like to withdraw from")
            #show all courses registered for
            #ask user to pick
            Show_all_My_Courses(conn, sID)
            cID = input("Enter the number of the course you would like to withdraw from:\n")
            #remove enrollment
            remove_enrollment(conn, sID, cID)
                
 
            #remove_enrollment(conn, sID)
        elif choice == 'S' or choice == 's':
            theCourse = input("Enter the course you are trying to find similar to: ")
            Show_similar_courses(conn, theCourse)
            continue
        elif choice == 'M' or choice == 'm':
            Show_all_My_Courses(conn, sID)
            continue
        elif choice == 'X' or choice == 'x':
            print("This registration window has ended. Goodbye")
            break
        else:
            print("Invalid response. Try again\n")
            continue
 
 
#----------------------------------------------------------------------------------------------------
 
 
#Establish Connection to SQL database
db = r"CourseProjectDB.db"
conn = sqlite3.connect(db)
c = conn.cursor()
 
 
# def create_tables(conn, SchemaTables, Schemas):
 
#     def get_Schema(Schema):
#         str = ""
 
#         for schema in Schema:
#             str+=schema
#             return str
 
#     if conn is None:
#         print("NO DB DETECTED....CREATING TABLES")
 
#         try:
#             createTableSql = "CREATE TABLE IF NOT EXISTS "
#             c=conn.cursor()
#             for i in range(0,len(SchemaTables)-1):
#                 cursor_string = createTableSql + get_Schema(Schemas[i]) + get_Schema(SchemaTables[i])
#                 c.execute(cursor_string)
 
#         except Error as e:
#             print(e)
 
 
# Create tables if no db is detected
# Enrolled = ("(","sID integer, ", "cID integer,",
#         "FOREIGN KEY (sID) REFERENCES Students (sID)",
#         "FOREIGN KEY (cID) REFERENCES Courses (cID)",");")
 
# Courses = ("(","cID integer PRIMARY KEY, ", 
#         "cName text, ", 
#         "numCredits integer",");")
 
# Students = ("(","sID integer PRIMARY KEY, ", 
#             "sName text",");")
 
 
# TableNames = ("Enrolled", "Courses", "Students")
# Tables = (Enrolled, Courses, Students)
 
 
# # Create Students, Enrolled, and Courses tables, if they don't already exist.
# create_tables(conn, Tables, TableNames)
 
 
 
entryMessage = "Welcome to the Student Registration"
print(entryMessage)
while True:
    #prompt user to enter -1 or their id
    usersID = input(("\nEnter your student ID or -1 if you are a new student"))
    if int(usersID) == -1:
        sName = input("Enter your name: ")
        Create_student(conn, usersID, sName)
        break
        
        
    else:
        if check_if_sID_exists(conn, usersID):
            sName = Get_studentsName(conn, usersID)
            print("Welcome back, " + sName)
            #userChoice = input("\nEnter a command:")
            student_menu(usersID)
        else:
              print("There is not student with that ID number. Try again.")
              continue
        break
 
conn.close()

Welcome to the Student Registration

Enter your student ID or -1 if you are a new student98742
There is not student with that ID number. Try again.

Enter your student ID or -1 if you are a new student-1
Enter your name: bill

Enter a command:x
This registration window has ended. Goodbye
